In [22]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import gensim
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from datasets import load_dataset
from gensim.models import KeyedVectors
import numpy as np
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from scipy.spatial import distance
from gensim.models.keyedvectors import KeyedVectors
from sklearn.metrics import mean_squared_error

In [23]:
model_W2V = KeyedVectors.load_word2vec_format('/kaggle/input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin', binary=True, limit=100000)

In [24]:
model_W2V.vector_size

300

In [25]:
print(f"Gensim Version: {gensim.__version__}")

Gensim Version: 4.3.2


In [26]:
# Load the English version of the STSB dataset
dataset = load_dataset("stsb_multi_mt", "en")

train_df = dataset['train'].to_pandas()
eval_df = dataset['dev'].to_pandas()
test_df = dataset['test'].to_pandas()

train_df['similarity_score'] /= 5.0
eval_df['similarity_score'] /= 5.0
test_df['similarity_score'] /= 5.0

In [27]:
def preprocess_text(text):
    
    tokens = word_tokenize(text)

    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.lower() not in stop_words]

    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

In [28]:

def compute_similarity(text1, text2):

    preprocessed_text1 = preprocess_text(text1)
    preprocessed_text2 = preprocess_text(text2)

    # converting to embeddings 
    embeddings1 = [model_W2V[word] for word in preprocessed_text1.split() if word in model_W2V]
    embeddings2 = [model_W2V[word] for word in preprocessed_text2.split() if word in model_W2V]

    # average embedding 
    avg_embedding1 = np.mean(embeddings1, axis=0) if embeddings1 else None
    avg_embedding2 = np.mean(embeddings2, axis=0) if embeddings2 else None

    if avg_embedding1 is not None and avg_embedding2 is not None:
        similarity = cosine_similarity([avg_embedding1], [avg_embedding2])[0][0]
    else:
        similarity = 0.0  

    return similarity


In [29]:
sim_score1 = compute_similarity("I had a bad day", "I had so much fun")
sim_score2 = compute_similarity("I had a bad day", "Everything was terrible today")

print("Similarity score 1:", sim_score1)
print("Similarity score 2:", sim_score2)

Similarity score 1: 0.38837838
Similarity score 2: 0.31493947


In [30]:
test_df['computed_similarity'] = test_df.apply(lambda row: compute_similarity(row['sentence1'], row['sentence2']), axis=1)

In [31]:
mse = mean_squared_error(test_df['similarity_score'], test_df['computed_similarity'])
print("Mean Squared Error (MSE):", mse)

Mean Squared Error (MSE): 0.10373513436930316
